In [1]:
import pandas as pd
import unidecode
import difflib
import pprint

import sys

file = '..\\datasets\\SNCF-opendata\\incidents-securite.json'
outputFile = '..\\outputs\\incidents-securite-reformated.xlsx'
rawData = pd.read_json(file)

The input data was downloaded from the [SNCF Open Data site](https://data.sncf.com/explore/), more precesily, the [Security Incidents dataset](https://data.sncf.com/explore/dataset/incidents-securite/table/?sort=date)

In [2]:
allData = pd.DataFrame( dict(rawData.fields) ).transpose()
allData['id'] = rawData.recordid

In [3]:
allData.describe()

,commentaires,date,esr,localisation,type,id
count,2527,2527,2527,2526,2522,2527
unique,2523,1070,3,1769,129,2527
top,Le chantier de libération a lieu sur voie 2 du...,2014-08-27,non,Miramas,Franchissement de signal,9e6081ecda230539782e5dd2d12f0cf1f28426c9
freq,2,12,1553,21,270,1


We will be first interested by the column 'type', by analysing the unique values of this column we observe that it contains in some cases, two or more values for the same type, e.g., 'Dérive' and 'dérive', 'Franchissement de Signal' and 'Franchissement de signal', 'Présence d'obstacle sur voie" and 'Obstacle sur voie', etc. We need to fix this.

In [4]:
#print(allData['type'].unique())

The objective is to replace the concerned values for the 'right' one, in other words, to create a dictionary.<br>
To save time, a previously obtained dictionary can be used, just set the ***usePredefinedDict*** variable to ***True***

In [5]:
usePredefinedDict = True

if usePredefinedDict:
    # The following dict was obtained after reviewing the data some times 
    specDict = {
        "Defaillance Voie" : "Defaillance voie",
        "Collision par prise en écharpe" : "Collision par prise en echarpe",
        "Défaillance matériel" : "Defaillance materiel",
        "Ouverture par un tiers d’une porte en marche" : "Ouverture par un tiers d'une porte en marche",
        "Dégagement intempestif du domaine fermé" : "Degagement intempestif du domaine ferme",
        "Défaillance Voie" : "Defaillance voie",
        "Circulation de véhicule non autorisée" : "Circulation de vehicule non autorisee",
        "Reception sur voie non-electrifiée" : "Reception sur voie non-electrifiee",
        "Collision contre obstacle à un passage à niveau" : "Collision contre obstacle a un passage a niveau",
        "Autorisation de départ sans autorisation" : "Circulation sans autorisation",
        "Point limite d’un pantographe dépassé par une circulation électrique" : "Point limite d'un pantographe depasse par une circulation electrique",
        "Défaut ayant conduit à un risque grave" : "Defaut ayant conduit a un risque grave",
        "Collision contre un obstacle a un passage a niveau" : "Collision contre obstacle a un passage a niveau",
        "Collision contre un obstacle à un passage à niveau" : "Collision contre obstacle a un passage a niveau",
        "Péremption du potentiel kilométrique" : "Peremption du potentiel kilometrique",
        "Réalimentation intempestive des caténaires" : "Realimentation intempestive des catenaires",
        "Nez à nez" : "Nez a nez",
        "Formation intempestive d'un itinéraire" : "Modification intempestive d'itineraire",
        "Quasi accident de travail" : "Accident du travail",
        "Collision contre obstacle et deraillement" : "Collision contre obstacle",
        "Délivrance d’un bulletin de franchissement sans vérification" : "Autorisation de franchissement sans verification",
        "derive" : "Derive",
        "Désordre d’ouvrage en terre" : "Desordre ouvrage d'art",
        "Arrêt prolongé d’un train de voyageurs dans un tunnel de plus de 1 km" : "Arret prolonge d'un train de voyageurs dans un tunnel de plus de 1 km",
        "Dérangement des installations de sécurité" : "Derangement des installations de securite",
        "Engagement de gabarit suite à désemparement de pièce" : "Engagement de gabarit suite a desemparement de piece",
        "nez à nez" : "Nez a nez",
        "Desordre d'ouvrage d'art" : "Desordre ouvrage d'art",
        "Désordre ouvrage d’art" : "Desordre ouvrage d'art",
        "Dégagement de fumée sur matériel roulant" : "Degagement de fumee sur materiel roulant",
        "reception sur voie non equipee" : "Reception sur voie secondaire",
        "Erreur grave de procédure" : "Erreur de procedure",
        "Accident de personne" : "Accident de voyageur",
        "incident grave de signalisation" : "Incident grave de signalisation",
        "Avarie au materiel" : "Non conformite du materiel roulant",
        "Erreur grave de procedure" : "Erreur de procedure",
        "Intempéries" : "Intemperies",
        "Déraillement" : "Deraillement",
        "Desordre d'ouvrage en terre" : "Desordre ouvrage d'art",
        "Autorisation de depart sans autorisation" : "Circulation sans autorisation",
        "Engagement intempestif d'une circulation sous catenaire consignee" : "Engagement intempestif d'une circulation sur une voie protegee",
        "Collision entre trains ou éléments de trains" : "Collision entre trains ou elements de trains",
        "Porte non vérroullée" : "Porte non verroullee",
        "Chute de voyageur" : "Accident de voyageur",
        "Delivrance d'un bulletin de franchissement sans verification" : "Autorisation de franchissement sans verification",
        "Incident de marchandise dangereuse" : "Marchandises dangereuses",
        "Défaillance d'ouvrage" : "Defaillance d'ouvrage",
        "Restitution DFV sans dégagement" : "Restitution DFV sans degagement",
        "Matières dangereuses" : "Marchandises dangereuses",
        "Erreur de procédure" : "Erreur de procedure",
        "incident caténaire" : "Incident catenaire",
        "Formation intempestive d'un itineraire" : "Modification intempestive d'itineraire",
        "Presence d'obstacle sur voie" : "Obstacle sur voie",
        "Avarie au matériel" : "Non conformite du materiel roulant",
        "Modification d'itinéraire sous un train" : "Modification d'itineraire sous un train",
        "Raté de fermeture à un passage à niveau" : "Rate de fermeture a un passage a niveau",
        "Ouverture par un tiers d’une porte de TGV en marche" : "Ouverture par un tiers d'une porte en marche",
        "Réception sur voie secondaire" : "Reception sur voie secondaire",
        "Avarie matériel roulant" : "Non conformite du materiel roulant",
        "Talonnage d’aiguille ou bivoie" : "Talonnage d'aiguille ou bivoie",
        "Depart sans autorisation" : "Circulation sans autorisation",
        "Evénement de sécurité sans boucle de rattrapage" : "Evenement de securite sans boucle de rattrapage",
        "Désordre d'ouvrage en terre" : "Desordre ouvrage d'art",
        "Penetration intempestive sur domaine ferme" : "Reception intempestive sur voie occupee",
        "Non respect du point d'arrêt" : "Non respect du point d'arret",
        "Evènement de sécurité sans boucle de rattrapage" : "Evenement de securite sans boucle de rattrapage",
        "Départ sans autorisation" : "Circulation sans autorisation",
        "Autorisation de franchissement sans vérification" : "Autorisation de franchissement sans verification",
        "robinet de suspension isolé" : "robinet de suspension isole",
        "Avarie materiel roulant" : "Non conformite du materiel roulant",
        "Matieres dangereuses" : "Marchandises dangereuses",
        "Accident de marchandise dangereuse" : "Marchandises dangereuses",
        "Défaut d’immobilisation" : "Defaut d'immobilisation",
        "Modification intempestive d'itinéraire" : "Modification intempestive d'itineraire",
        "Circulation d’un Transport Exceptionnel sans annonce" : "Circulation d'un Transport Exceptionnel sans annonce",
        "Dépassement de vitesse supérieur à 40km/h" : "Depassement de vitesse superieur a 40km/h",
        "Présence de voyageurs sur voie" : "Presence de voyageurs sur voie",
        "Non conformité du matériel roulant" : "Non conformite du materiel roulant",
        "Mauvaise matérialisation du point de dégagement" : "Mauvaise materialisation du point de degagement",
        "Appareil de voie mal disposé" : "Appareil de voie mal dispose",
        "Acte de Malveillance" : "Acte de malveillance",
        "Perte d'organe du materiel roulant" : "Non conformite du materiel roulant",
        "Sortie intempestive ou engagement d’un domaine fermé" : "Sortie intempestive ou engagement d'un domaine ferme",
        "nez a nez" : "Nez a nez",
        "Engagement intempestif d’une circulation sous caténaire consignée" : "Engagement intempestif d'une circulation sur une voie protegee",
        "dérive" : "Derive",
        "accident du travail" : "Accident du travail",
        "Dérive et collision" : "Derive et collision",
        "Pénétration intempestive sur domaine fermé" : "Reception intempestive sur voie occupee",
        "reception sur voie non équipée" : "Reception sur voie secondaire",
        "Engagement de gabarit avec collision" : "Engagement de gabarit",
        "Mise en œuvre de travaux en dehors du domaine fermé" : "Mise en oeuvre de travaux en dehors du domaine ferme",
        "Défaut sur aiguille" : "Defaut sur aiguille",
        "Défaillance voie" : "Defaillance voie",
        "incident catenaire" : "Incident catenaire",
        "Matieres Dangereuses" : "Marchandises dangereuses",
        "Franchissement de Signal" : "Franchissement de signal",
        "Présence d'obstacle sur voie" : "Obstacle sur voie",
        "Incident caténaire" : "Incident catenaire",
        "Non respect du point d’arrêt" : "Non respect du point d'arret",
        "Erreur d’itinéraire" : "Erreur d'itineraire",
        "Engagement intempestif d’une circulation sur une voie protégée" : "Engagement intempestif d'une circulation sur une voie protegee",
        "Incident grave de passage à niveau" : "Incident grave de passage a niveau",
        "Réception intempestive sur voie occupée" : "Reception intempestive sur voie occupee",
        "Matières Dangereuses" : "Marchandises dangereuses",
        "Désordre d’ouvrage d'art" : "Desordre ouvrage d'art",
        "Défaut géométrie voie" : "Defaut geometrie voie",
        "Arrêt hors de quai de tout ou partie d’un train de voyageurs" : "Arret hors de quai de tout ou partie d'un train de voyageurs",
        "Manoeuvre vers non électrifiée" : "Manoeuvre vers non electrifiee",
        "Perte d’organe du matériel roulant" : "Non conformite du materiel roulant",
        "Frein serré" : "Frein serre",
        "Electrisation" : "Electrocution",
        "Dérive" : "Derive",
        "Déshuntage" : "Deshuntage",
        "porte ouverte en ligne" : "Porte ouverte en ligne",
        "Boîte chaude" : "Boite chaude",
        "Expédition sans ordre" : "Expedition sans ordre",
        "Déraillement sans engagement de la voie principale" : "Deraillement sans engagement de la voie principale",
        "Evolution vers une voie occupée" : "Evolution vers une voie occupee",
        "Ouverture par un tiers d'une porte de TGV en marche" : "Ouverture par un tiers d'une porte en marche"
}
else: 
    specDict = {}


In [6]:
def myPrintDict(theDict):
    print('\nTheDict:\n{')
    for i in theDict:
        print('\"{}\" : \"{}\",'.format(i, theDict[i]))
    print('}')

def searchRecInDict(theWord, theDict): 
    #Performs a recursive search of a word within a dict
    if theWord in theDict:
        try:
            return searchRecInDict(theDict[theWord], theDict)
        except RecursionError as re:
            message = 'Invalid Dictionary! It contains an infinite loop e.g., A->B, B->C, C->A, check item \"{}\" '
            message += 'This will probably cause errors'
            print(message.format(theWord))
    else:
        return theWord

Let's try the code below for possible infinite loops, It should raise an error!

In [7]:
testDict = {'A' : 'B',
        'B' : 'C',
        'C' : 'A',
        'E': 'F'}
searchRecInDict('A', testDict)  # should raise error

Invalid Dictionary! It contains an infinite loop e.g., A->B, B->C, C->A, check item "B" This will probably cause errors


In [8]:
def nomalizeListAndUpdateDict(theList, theDict):
    for idx in range(len(theList)):
        unicodeVersion = unidecode.unidecode(theList[idx])
        if theList[idx] != unicodeVersion:
            theDict[theList[idx]] = unicodeVersion
            theList[idx] = unicodeVersion
    return theList, theDict

def fastUpdateListAndDict(theList, theDict):
    for key, value in theDict.items():
        foundValue = searchRecInDict(key, theDict)
        if value != foundValue:
            theDict[key] = foundValue

    for idx in range(len(theList)):
        theTerm = theList[idx]
        if theTerm in theDict:
            theList[idx] = theDict[theTerm]

    return list(set(theList)), theDict

def performUserChoice(theList, termIdx, theDict, optionList):
    message = 'Possible conflict of \033[4m\"{}\"\033[0m, select which option to use\n'.format(theList[termIdx])
    message += '[0] = Keep this value, '
    for (i, item) in enumerate(optionList):
        message += '[' + str(i+1) + '] = \"' + item + '\", '
#         if item in theDict.values():   #This could be used to 'suggest' an option that was previously used
#             message += '\"*, '
#         else:
#             message += '\", '
    print(message)
    inputMessage = 'Enter your choice (number between 0 and {} or -1 to abort): '.format( len(optionList) )
    usrChoice = int(input( inputMessage ) )
    if usrChoice < 1:
        return True if usrChoice == -1 else False, theList, theDict
    #TODO: put code here to verify if input is integer within corresponding range
    theDict[theList[termIdx]] = optionList[usrChoice-1] 
    theList[termIdx] = optionList[usrChoice-1]
    return False, theList, theDict
    
def oneStep(theList, termIdx, theDict, p_cutoff = 0.7, p_n = 4):
#Returns T/F if the execution must be interrupted, the updated list and dictionary 
    similarTypes = difflib.get_close_matches(theList[termIdx], theList[:termIdx] + theList[termIdx+1:], 
                         n = p_n, cutoff = p_cutoff )

    similarTypes, theDict = fastUpdateListAndDict(similarTypes, theDict)
    similarTypes = list(filter(lambda a: a != theList[termIdx], similarTypes))
        
    if len(similarTypes) < 1:
        return False, theList, theDict 
    else:
        #optionList = [theTerm] + similarTypes
        return performUserChoice(theList, termIdx, theDict, similarTypes)

def assistedCategoricalNormalization(theList, theDict = {}, normToUnicode = False):
    begIterMsg = 'Performing similarity search, current cutoff coefficient: {}'
    endIterMsg = 'Completed! No further similarities were found. In total, {} reassignements were made.'
    
    cutoff_coef = 0.8 if len(theDict) > 0 else 0.6
    
    if any(not isinstance(x, str) for x in theList):
        print('The provided list contains at least one element that is not a string! Aborting operation')
        return theList, theDict
    
    if normToUnicode:
        print('Normalizing the initial list to Unicode')
        theList, theDict = nomalizeListAndUpdateDict(theList, theDict)

    
    theList, theDict = fastUpdateListAndDict(theList, theDict)
    while True:
        initListLen = len(theList)
        print(begIterMsg.format(cutoff_coef))
        for idx in range(len(theList)):
            abortExec, theList, theDict = oneStep(theList, idx, theDict, cutoff_coef)
            if abortExec:
                print('Aborting current execution')
                return theList, theDict
        
        theList, theDict = fastUpdateListAndDict(theList, theDict)
        
        reassignements = initListLen - len(theList)
        print(endIterMsg.format(reassignements))
        if reassignements == 0 or input('Do you wish to perform another pass? (y / n') == 'n':
            break
        else:
            cutoff_coef += (1 - cutoff_coef) * 25/100
            
    return theList, theDict

In [9]:
#The purpose of this cell is for testing the different functions
testDict = {'Hallo' : 'Hello',
        'Big' : 'Bigger',
        'Colr' : 'Colour',
        'Colour' : 'Color',
        'Extremist': 'Extreme'}

testList = ['Hallo', 'Hello', 'Biger', 'Colr', 'Color', 'Extrem', 'Extreme', 'Extremist']
#rep, newTerm, newDict = oneStep(testList, 0, testDict)
#print(oneStep(testList, 1, testDict))
#print( assistedCategoricalNormalization(testList, testDict) )

In [10]:
rawTypes = allData['type'].unique()

if any(not isinstance(x, str) for x in rawTypes):
    rawTypes = [ x for x in rawTypes if isinstance(x, str) ]
    rawTypes.append('Undefined')

normalizedTypes, newSpecDict = assistedCategoricalNormalization(rawTypes, specDict, True)

Normalizing the initial list to Unicode
Performing similarity search, current cutoff coefficient: 0.8
Completed! No further similarities were found. In total, 0 reassignements were made.


Use the following code to print the obtained dict and put it in previously in the notebook to avoid manually entering the choices again

In [11]:
printTypesAndDict = False
if printTypesAndDict:
    print(str(len(normalizedTypes)) + ' types\n', normalizedTypes, '\n\n')
    myPrintDict(newSpecDict)
    

In [12]:
# print(len(newSpecDict))
# normalizedTypes, newSpecDict = fastUpdateListAndDict(normalizedTypes, newSpecDict)
# print(len(newSpecDict))

allData = allData.replace({"type": newSpecDict})

In [13]:
topMost = len(allData['type'].unique())
#opMost = 29

incByTypeESR_df = pd.DataFrame( allData.groupby( ['type','esr'] ).size().reset_index(name = 'count') )
incByTypeESR_df = incByTypeESR_df.pivot_table(index='type', columns='esr', aggfunc=sum).fillna(0)

incByTypeESR_df['esr_oui_c'] = incByTypeESR_df['count']['oui']
incByTypeESR_df['esr_non_c'] = incByTypeESR_df['count']['non']
incByTypeESR_df['esr_und_c'] = incByTypeESR_df['count']['ND']
incByTypeESR_df['tot_count'] = incByTypeESR_df['esr_oui_c'] + incByTypeESR_df['esr_non_c'] + incByTypeESR_df['esr_und_c']

incByTypeESR_df = incByTypeESR_df.drop(columns=['count'])

# TODO: use sort_values instead so more than one column can be used
incByTypeESRMost_df = incByTypeESR_df.nlargest(topMost, 'esr_oui_c') 

theRest = incByTypeESR_df.drop(incByTypeESRMost_df.index).sum()
theRest_df = pd.DataFrame(theRest).transpose()
theRest_df.insert(0, 'type', 'Autres')

incByTypeESRMost_df = incByTypeESRMost_df.reset_index().append(theRest_df, ignore_index=True)

incByTypeESRMost_df

,type,esr_oui_c,esr_non_c,esr_und_c,tot_count
esr,,,,,
0,Franchissement de signal,100.0,172.0,0.0,272.0
1,Defaillance voie,76.0,42.0,0.0,118.0
2,Erreur de procedure,72.0,72.0,0.0,144.0
3,Depassement de vitesse superieur a 40km/h,65.0,6.0,0.0,71.0
4,Defaut geometrie voie,54.0,13.0,0.0,67.0
5,Rate de fermeture a un passage a niveau,50.0,16.0,0.0,66.0
6,Deraillement,50.0,53.0,0.0,103.0
7,Porte ouverte en ligne,47.0,41.0,0.0,88.0
8,Derive,39.0,34.0,0.0,73.0


In [14]:
# Need to C&P the the results in a notepad for printing..
#print(incByTypeAll_df)
pd.options.display.max_colwidth = 100
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 100):
    print(incByTypeESRMost_df.sort_values(['esr_oui_c', 'tot_count'], ascending= False).to_string(index=False))

type esr_oui_c esr_non_c esr_und_c tot_count
                                                                                                             
                                             Franchissement de signal     100.0     172.0       0.0     272.0
                                                     Defaillance voie      76.0      42.0       0.0     118.0
                                                  Erreur de procedure      72.0      72.0       0.0     144.0
                            Depassement de vitesse superieur a 40km/h      65.0       6.0       0.0      71.0
                                                Defaut geometrie voie      54.0      13.0       0.0      67.0
                                                         Deraillement      50.0      53.0       0.0     103.0
                              Rate de fermeture a un passage a niveau      50.0      16.0       0.0      66.0
                                               Porte ouverte en ligne      

The next part consist on trying to automatically extract information from the 'commentaires' column, the objective is to structurate this data by associating a 'cause factor' and a 'responsability' to each incident 

In [15]:
writeNewXLSX = True
if (writeNewXLSX):
    toExport = allData[['id', 'date', 'localisation', 'type', 'esr', 'commentaires']].sort_values('date')
    causeFactorCols = ['CF_Communication', 'CF_Distraction', 'CF_Familiarity', 'CF_Fatigue', 'CF_QoInformation', 
           'CF_Perception', 'CF_QoProcedure', 'CF_SafetyCulture', 'CF_Supervision', 'CF_InfraFailure', 
           'CF_RollStockFailure', 'CF_SystemDesign', 'CF_Training', 'CF_Workload', 'CF_External']
    responsabilityCols = ['R_Driver', 'R_OnBoardStaff', 'R_Signaller', 'R_Dispatcher', 'R_SiteGuard', 
            'R_SiteStaff', 'R_Shunter', 'R_Passenger', 'R_External', 'R_Attack']
    newCols = causeFactorCols + responsabilityCols
    toExport = pd.concat([toExport, pd.DataFrame(columns = newCols )], sort = False)
    writer = pd.ExcelWriter(outputFile)
    toExport.to_excel(writer,'allData', index = False)
    writer.save()